# KNN

k개의 주변 이웃 중 가장 많은 값으로 분류

In [1]:
import pandas as pd
import seaborn as sns

In [6]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()   ##age,embarked,embarked_town,deck 변수에 결측치 존재 -> deck은 제거, embarked=embarked_town이므로 하나 제거

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [8]:
df=df.drop(['deck','embark_town'],axis=1) 
df=df.dropna(subset=['age'],axis=0)      #결측치 행 삭제

In [9]:
df['embarked'].value_counts()

S    554
C    130
Q     28
Name: embarked, dtype: int64

In [11]:
df['embarked'].fillna('s',inplace=True)   #결측치 최빈갑으로 처리

In [13]:
ndf=df[['survived','pclass','sex','age','sibsp','parch','embarked']]   #변수 선택

In [14]:
#원핫인코딩
onehot_sex=pd.get_dummies(ndf['sex'])
ndf=pd.concat([ndf,onehot_sex],axis=1)

onthot_embarked=pd.get_dummies(ndf['embarked'])
ndf=pd.concat([ndf,onthot_embarked],axis=1)

ndf.drop(['sex','embarked'],axis=1,inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,C,Q,S,s
0,0,3,22.0,1,0,0,1,0,0,1,0
1,1,1,38.0,1,0,1,0,1,0,0,0
2,1,3,26.0,0,0,1,0,0,0,1,0
3,1,1,35.0,1,0,1,0,0,0,1,0
4,0,3,35.0,0,0,0,1,0,0,1,0


In [16]:
ndf.drop(['s'],axis=1,inplace=True)

In [17]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,C,Q,S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [22]:
X=ndf[['pclass','age','sibsp','parch','female','male','C','Q','S']]
y=ndf['survived']

from sklearn import preprocessing
X=preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split
X_train,y_train,X_test,y_test=train_test_split(X,y,test_size=0.2,random_state=111)